<a href="https://colab.research.google.com/github/arminaria456/classifier/blob/main/classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import re
import pandas as pd
from collections import Counter

# Load the dataset
sms_spam = pd.read_csv('spam.csv', names=['Label', 'SMS'])

# Split the dataset into training and test sets
training_test_index = round(len(sms_spam) * 0.8)
training_set = sms_spam[:training_test_index].reset_index(drop=True)
test_set = sms_spam[training_test_index:].reset_index(drop=True)

# Create the vocabulary
vocabulary = []
for sms in training_set['SMS']:
    sms = re.sub('\W',' ' , sms)
    sms = sms.lower().split()
    vocabulary.extend(sms)
vocabulary = list(set(vocabulary))

# Calculate the prior probabilities
p_spam = len(training_set[training_set['Label'] == 'pam']) / len(training_set)
p_ham = len(training_set[training_set['Label'] == 'ham']) / len(training_set)

# Calculate the likelihood probabilities
n_spam = len(training_set[training_set['Label'] == 'pam'])
n_ham = len(training_set[training_set['Label'] == 'ham'])

parameters_spam = {}
parameters_ham = {}

for word in vocabulary:
    spam_count = 0
    ham_count = 0
    for sms in training_set['SMS']:
        sms = re.sub('\W',' ' , sms)
        sms = sms.lower().split()
        if word in sms:
            if training_set.loc[sms_spam.index[sms], 'Label'] == 'pam':
                spam_count += 1
            else:
                ham_count += 1
    parameters_spam[word] = (spam_count + 1) / (n_spam + len(vocabulary))
    parameters_ham[word] = (ham_count + 1) / (n_ham + len(vocabulary))

def classify(message):
    message = re.sub('\W', '', message)
    message = message.lower().split()
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
    if p_ham_given_message > p_spam_given_message:
        return 'Ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'Spam'
    else:
        return 'Equal probabilities, have a human classify this!'

# Test the classifier
classify('WINNER!! This is the secret code to unlock the money: C3421.')

FileNotFoundError: [Errno 2] No such file or directory: 'spam.csv'